# DRL Project - MountainCar-v0

## Components

### Libraries

In [1]:
!pip install keras-rl2

In [2]:
from gym.envs.classic_control.mountain_car import MountainCarEnv
from rl.policy import GreedyQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger
from rl.agents import DQNAgent
from colabgymrender.recorder import Recorder
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import json
import gym

print(f'tensorflow version = {tf.__version__}')
print(tf.config.list_physical_devices('GPU'))
print()

tensorflow version = 2.5.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



In [3]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

Requirement already up-to-date: colabgymrender in /usr/local/lib/python3.7/dist-packages (1.0.9)


In [4]:
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

### Environment

In [5]:
class MountainCarWithNoiseEnv(MountainCarEnv):
    def __init__(self, goal_velocity=0, noise_std=0):
        super().__init__(goal_velocity)
        self.noise_std = noise_std

    def step(self, action):
        state, reward, done, info = super().step(action)
        
        if self.noise_std > 0:
            state[0] += np.random.normal(0, self.noise_std)
            state[1] += np.random.normal(0, self.noise_std)
        
        return state, reward, done, info

### Neural Network

In [6]:
def get_model(env):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(1,) + env.observation_space.shape))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(env.action_space.n))
    
    return model

### Agent

In [7]:
def run_experiment(experiment, policy, noise_std):   
    results_file_name = get_results_file_name(experiment)
    
    try:
        env = MountainCarWithNoiseEnv(noise_std=noise_std)
        
        env.seed(0)
        np.random.seed(0)
        
        model = get_model(env)
        
        agent = DQNAgent(model=model,
                         nb_actions=env.action_space.n, 
                         memory=SequentialMemory(limit=50000, window_length=1), 
                         nb_steps_warmup=50, 
                         target_model_update=1e-2, 
                         policy=policy)
        
        callbacks=[FileLogger(results_file_name)]
        
        agent.compile(keras.optimizers.Adam(lr=1e-3), metrics=['mae'])
        agent.fit(env, nb_steps=100000, nb_max_episode_steps=200, callbacks=callbacks,
                  visualize=False, verbose=1) # FIXME
                
        # input("Press enter to start to testing...")
        # agent.test(env, nb_episodes=5, visualize=True)
    finally:
        env.close()

### Plots

In [8]:
from google.colab import drive
drive.mount('/content/drive')

MAIN_DIR = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def get_results_file_name(experiment):
    return MAIN_DIR + experiment + '.json'

def exponential_smoothing(x, alpha):
    y = np.zeros_like(x)
    
    y[0] = x[0]
    
    for i in range(1, len(x)):
        y[i] = alpha*x[i] + (1-alpha)*y[i-1]
    
    return y

def plot_results(experiments, prefix=None):
    plt.figure()
    plt.title('Mean reward per episode')
    plt.xlabel('Episodes')
    plt.ylabel('Reward')
    for experiment in experiments:
        results_file_name = get_results_file_name(experiment)     
        with open(results_file_name, 'r') as f:
            results = json.load(f)
        episodes = results['episode']
        rewards = results['episode_reward']
        # rewards = exponential_smoothing(rewards, 0.01) # FIXME
        plt.plot(episodes, rewards, label=experiment)
    if len(experiments) > 1:
        plt.legend()
    plt.show()
    plt.savefig(MAIN_DIR + prefix + '_rewards.jpg')
    
    plt.figure()
    plt.title('Mean Q per episode')
    plt.xlabel('Episodes')
    plt.ylabel('Q')
    for experiment in experiments:
        results_file_name = get_results_file_name(experiment)     
        with open(results_file_name, 'r') as f:
            results = json.load(f)
        episodes = results['episode']
        q = results['mean_q']
        plt.plot(episodes, q, label=experiment)
    if len(experiments) > 1:
        plt.legend()
    plt.show()
    plt.savefig(MAIN_DIR + prefix + '_q.jpg')

## Experiments

### Without noise

#### Greedy

In [10]:
run_experiment('Greedy', GreedyQPolicy(), 0)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: -1.0000
50 episodes - episode_reward: -199.140 [-200.000, -157.000] - loss: 2.556 - mae: 20.148 - mean_q: -29.786

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: -1.0000
56 episodes - episode_reward: -177.232 [-200.000, -87.000] - loss: 5.226 - mae: 34.720 - mean_q: -51.295

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: -1.0000
63 episodes - episode_reward: -159.444 [-200.000, -86.000] - loss: 4.323 - mae: 33.924 - mean_q: -50.065

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: -1.0000
74 episodes - episode_reward: -134.622 [-200.000, -87.000] - loss: 2.879 - mae: 31.332 - mean_q: -46.095

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 93s 9ms/step - 

In [11]:
plot_results('Greedy')

TypeError: ignored

#### EpsGreedy eps=0.05

In [ ]:
run_experiment('EpsGreedy eps=0.05', EpsGreedyQPolicy(eps=0.05), 0)

In [ ]:
plot_results('EpsGreedy eps=0.05')

#### EpsGreedy eps=0.1

In [ ]:
run_experiment('EpsGreedy eps=0.1', EpsGreedyQPolicy(eps=0.1), 0)

In [ ]:
plot_results('EpsGreedy eps=0.1')

#### EpsGreedy eps=0.2

In [ ]:
run_experiment('EpsGreedy eps=0.2', EpsGreedyQPolicy(eps=0.2), 0)

In [ ]:
plot_results('EpsGreedy eps=0.2')

#### MaxBoltzmann eps=0.05

In [ ]:
run_experiment('MaxBoltzmann eps=0.05', MaxBoltzmannQPolicy(eps=0.05), 0)

In [ ]:
plot_results('MaxBoltzmann eps=0.05')

#### MaxBoltzmann eps=0.1

In [ ]:
run_experiment('MaxBoltzmann eps=0.1', MaxBoltzmannQPolicy(eps=0.1), 0)

In [ ]:
plot_results('MaxBoltzmann eps=0.1')

#### MaxBoltzmann eps=*0.2*

In [ ]:
run_experiment('MaxBoltzmann eps=0.2', MaxBoltzmannQPolicy(eps=0.2), 0)

In [ ]:
plot_results('MaxBoltzmann eps=0.2')

#### Total Results

In [ ]:
experiments = [
               'Greedy'
               'EpsGreedy eps=0.05',
               'EpsGreedy eps=0.1',
               'EpsGreedy eps=0.2',
               'MaxBoltzmann eps=0.05',
               'MaxBoltzmann eps=0.1',
               'MaxBoltzmann eps=0.2',
               ]
plot_results(experiments)